# [Data Preparing](https://github.com/CMACH508/DeepTrader?tab=readme-ov-file#data-preparing)

|                    File_name                     |                  shape                   |                  description                   |
| :----------------------------------------------: | :--------------------------------------: | :--------------------------------------------: |
|                 stocks_data.npy                  | [num_stocks, num_days, num_ASU_features] |       the inputs for asset scoring unit        |
|                 market_data.npy                  |       [num_days, num_MSU_features]       |     the inputs for marketing scoring unit      |
|                     ror.npy                      |          [num_stocks, num_days]          | rate of return file for calculating the return |
| relation_file (e.g. industry_classification.npy) |         [num_stocks, num_stocks]         |     the relation matrix used in GCN layer      |



In [38]:
import pandas as pd
TWII = "^TWII"
toptw = pd.read_excel(r'0050.xlsx')
TWII_STOCKS = [str(symbol) + '.TW' for symbol in toptw['Symbol']]
print(TWII_STOCKS)
print(f"Total stocks to process: {len(TWII_STOCKS)}")


# Define a small epsilon to avoid 0 values after normalization
EPSILON = 1e-6

['2330.TW', '2454.TW', '2317.TW', '2382.TW', '2308.TW', '2303.TW', '2891.TW', '3711.TW', '2881.TW', '2412.TW', '2886.TW', '2882.TW', '2884.TW', '1216.TW', '2885.TW', '3231.TW', '3034.TW', '2357.TW', '2002.TW', '2892.TW', '1303.TW', '5880.TW', '2379.TW', '1301.TW', '2890.TW', '3008.TW', '3037.TW', '2345.TW', '5871.TW', '3661.TW', '2880.TW', '2327.TW', '2883.TW', '2301.TW', '1101.TW', '2887.TW', '2207.TW', '4938.TW', '6669.TW', '1326.TW', '3045.TW', '2395.TW', '5876.TW', '2603.TW', '1590.TW', '2912.TW', '4904.TW', '2801.TW', '6505.TW', '2408.TW']
Total stocks to process: 50


In [39]:
import yfinance as yf

# Get the earliest available date for the TWII index
twii_ticker = yf.Ticker(TWII)
twii_history = twii_ticker.history(period="max")
if not twii_history.empty:
    twii_earliest_date = twii_history.index.min()
    print("TWII earliest date:", twii_earliest_date)
else:
    print("No historical data found for TWII.")

# Get the earliest available date for each stock in TWII_STOCKS\
# Create a Tickers object for multiple stocks
twii_tickers = yf.Tickers(" ".join(TWII_STOCKS))

# Dictionary to hold each stock's earliest date
stocks_earliest_dates = {}

for stock in TWII_STOCKS:
    ticker = twii_tickers.tickers[stock]
    stock_history = ticker.history(period="max")
    if not stock_history.empty:
        earliest_date = stock_history.index.min()
        stocks_earliest_dates[stock] = earliest_date
    else:
        stocks_earliest_dates[stock] = None

print("TWII Stocks earliest dates:")
for stock, date in stocks_earliest_dates.items():
    print(f"{stock}: {date}")

TWII earliest date: 1997-07-02 00:00:00+08:00
TWII Stocks earliest dates:
2330.TW: 2000-01-04 00:00:00+08:00
2454.TW: 2001-07-23 00:00:00+08:00
2317.TW: 1993-01-05 00:00:00+08:00
2382.TW: 2000-01-04 00:00:00+08:00
2308.TW: 2000-01-04 00:00:00+08:00
2303.TW: 2000-01-04 00:00:00+08:00
2891.TW: 2002-05-17 00:00:00+08:00
3711.TW: 2000-01-04 00:00:00+08:00
2881.TW: 2001-12-20 00:00:00+08:00
2412.TW: 2000-11-15 00:00:00+08:00
2886.TW: 2000-01-04 00:00:00+08:00
2882.TW: 2000-01-04 00:00:00+08:00
2884.TW: 2002-01-29 00:00:00+08:00
1216.TW: 2000-01-04 00:00:00+08:00
2885.TW: 2000-01-04 00:00:00+08:00
3231.TW: 2003-08-19 00:00:00+08:00
3034.TW: 2002-08-27 00:00:00+08:00
2357.TW: 2000-01-04 00:00:00+08:00
2002.TW: 2000-01-04 00:00:00+08:00
2892.TW: 2003-01-03 00:00:00+08:00
1303.TW: 2000-01-04 00:00:00+08:00
5880.TW: 2000-01-04 00:00:00+08:00
2379.TW: 2000-01-04 00:00:00+08:00
1301.TW: 2000-01-04 00:00:00+08:00
2890.TW: 2000-01-04 00:00:00+08:00
3008.TW: 2002-03-11 00:00:00+08:00
3037.TW: 2000-01

In [15]:
import datetime

for i in range(2000, 2019):
	stocks_to_remove = []
	target_date = datetime.date(i, 1, 10)
	for stock, dt in stocks_earliest_dates.items():
		if dt is None or dt.replace(tzinfo=None).date() > target_date:
			stocks_to_remove.append(stock)
	# print("Stocks to remove:", stocks_to_remove)
	print(str(i) + " " + str(len(stocks_to_remove)))


2000 22
2001 20
2002 16
2003 9
2004 7
2005 7
2006 6
2007 6
2008 6
2009 6
2010 5
2011 4
2012 3
2013 2
2014 2
2015 1
2016 1
2017 1
2018 0


In [32]:
import datetime

stocks_to_remove = []
target_date = datetime.date(2015, 1, 4)
for stock, dt in stocks_earliest_dates.items():
	if dt is None or dt.replace(tzinfo=None).date() > target_date:
		stocks_to_remove.append(stock)
print("Stocks to remove:", stocks_to_remove)
print(len(stocks_to_remove))

for stock in stocks_to_remove:
	if stock in TWII_STOCKS:
		TWII_STOCKS.remove(stock)

Stocks to remove: ['6669.TW']
1


In [40]:
TARGET_DIR = "."
# Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
# PERIOD = "10y"
START_DATE = "2015-01-01"
END_DATE = "2025-03-31"
# Open, High, Low, Close, Volume, Dividends, Stock Splits
ASSET_FEATURES = ["Open", "High", "Low", "Close", "Volume"]
MARKET_FEATURES = ["Open", "High", "Low", "Close"]

In [25]:
import pandas as pd

# Generate business days for the fixed date range
business_days = pd.bdate_range(start=START_DATE, end=END_DATE)
print("Total business days:", len(business_days))
print()

intervals = {
    "Training": ("2015/01/01", "2019/12/31"),
    "Validation": ("2020/01/01", "2022/12/31"),
    "Test": ("2023/01/01", "2025/03/31"),
}
for interval_name, (start_date, end_date) in intervals.items():
    interval_days = pd.bdate_range(start=start_date, end=end_date)
    start_idx = business_days.get_loc(interval_days[0])  # find the start index
    end_idx = business_days.get_loc(interval_days[-1])   # find the end index
    total_days = len(interval_days)  # calculate the total number of business days
    
    print(f"{interval_name}:")
    print(f"  Start Index = {start_idx}")
    print(f"  End Index = {end_idx}")
    print(f"  Total Business Days = {total_days}\n")


Total business days: 2673

Training:
  Start Index = 0
  End Index = 1303
  Total Business Days = 1304

Validation:
  Start Index = 1304
  End Index = 2086
  Total Business Days = 783

Test:
  Start Index = 2087
  End Index = 2672
  Total Business Days = 586



## stocks_data.npy
| File_name | Shape | Description |
| ---- | ---- | ---- |
|stocks_data.npy |	[num_stocks, num_days, num_ASU_features] |	the inputs for asset scoring unit |

In [34]:
import yfinance as yf
import numpy as np

stocks_data = []

# Process each stock
for stock in TWII_STOCKS:
    ticker = yf.Ticker(stock)
    # Download historical data for the given date range
    df = ticker.history(start=START_DATE, end=END_DATE)[ASSET_FEATURES]
    
    # Remove timezone information from the index to match the naive date_range
    df.index = df.index.tz_localize(None)
    # Reindex to the fixed business day range
    df = df.reindex(business_days)
    
    # Check missing values: count total NaN / total (rows x columns)
    total_cells = df.shape[0] * df.shape[1]
    nan_count = df.isna().sum().sum()
    nan_ratio = nan_count / total_cells
    if nan_ratio > 0.1:
        print(f"Warning: {stock} has {nan_ratio:.2%} missing values.")
    
    # Fill missing values using forward fill, then backward fill if necessary
    df.replace(0, np.nan, inplace=True)
    df.ffill(inplace=True)
    df.bfill(inplace=True)
    
    # # normalization per column
    # for feature in ASSET_FEATURES:
    #     min_val = df[feature].min()
    #     max_val = df[feature].max()
    #     # Standard min-max normalization, then shift the scale
    #     df[feature] = ((df[feature] - min_val) / (max_val - min_val)) * (1 - EPSILON) + EPSILON
    
    stocks_data.append(df)

# Convert list of DataFrames to numpy array with shape (number_of_stocks, number_of_days, number_of_features)
stocks_data_np = np.stack([df.values for df in stocks_data])
print(stocks_data_np.shape)

np.save(f"{TARGET_DIR}/stocks_data.npy", stocks_data_np)

(49, 2673, 5)


# market_data.npy
| File_name | Shape | Description |
| ---- | ---- | ---- |
| market_data.npy | [num_days, num_MSU_features] | the inputs for marketing scoring unit |

In [41]:
# Download TWII historical data for the given date range
ticker = yf.Ticker(TWII)
market_data = ticker.history(start=START_DATE, end=END_DATE)[MARKET_FEATURES]

# save to csv
market_data.to_csv(f"{TARGET_DIR}/^TWII.csv")

In [ ]:
# Remove timezone information from the index so it matches the naive date_range
market_data.index = market_data.index.tz_localize(None)
# Reindex the DataFrame to the fixed business day range
market_data = market_data.reindex(business_days)

# Check missing values: calculate total cells and NaN ratio
total_cells = market_data.shape[0] * market_data.shape[1]
nan_count = market_data.isna().sum().sum()
nan_ratio = nan_count / total_cells
if nan_ratio > 0.1:
    print(f"Warning: TWII has {nan_ratio:.2%} missing values.")

# Fill missing values using forward fill, then backward fill if necessary
market_data.replace(0, np.nan, inplace=True)
market_data.ffill(inplace=True)
market_data.bfill(inplace=True)

# # Normalize each column (feature)
# for feature in MARKET_FEATURES:
#     min_val = market_data[feature].min()
#     max_val = market_data[feature].max()
#     # Standard min-max normalization, then shift the scale
#     market_data[feature] = ((market_data[feature] - min_val) / (max_val - min_val)) * (1 - EPSILON) + EPSILON
    
# Convert the DataFrame to a numpy array and save it
market_data_np = market_data.to_numpy()
print(market_data_np.shape)
np.save(f"{TARGET_DIR}/market_data.npy", market_data_np)

(2673, 4)


# ror.py
| File_name | Shape | Description |
| ---- | ---- | ---- |
| ror.npy | [num_stocks, num_days] | rate of return file for calculating the return|

In [36]:
import yfinance as yf
import pandas as pd
import numpy as np

# Dictionary to collect rate of return series for each stock
ror_data = {}

# Process each stock in TWII_STOCKS
for stock in TWII_STOCKS:
    ticker = yf.Ticker(stock)
    # Download historical data for the given date range
    data = ticker.history(start=START_DATE, end=END_DATE)
    
    # Remove timezone info from the index so that it matches the naive date_range
    data.index = data.index.tz_localize(None)
    
    # Reindex to the fixed business day range (if a date is missing, NaN is inserted)
    data = data.reindex(business_days)
    
    # Check missing values: count total cells and compute NaN ratio
    total_cells = data.shape[0] * data.shape[1]
    nan_count = data.isna().sum().sum()
    nan_ratio = nan_count / total_cells
    if nan_ratio > 0.1:
        print(f"Warning: {stock} has {nan_ratio:.2%} missing values in raw data.")
    
    # Fill missing values using forward fill, then backward fill if necessary
    data.ffill(inplace=True)
    data.bfill(inplace=True)
    
    # Compute daily rate of return: (Close / Open) - 1.0
    ror_series = data["Close"] / data["Open"] - 1.0
    ror_data[stock] = ror_series

# Create a DataFrame with rows = stocks, columns = dates
ror_df = pd.DataFrame(ror_data).transpose()
print(ror_df.shape)  # Should be (num_stocks, num_days)

# Save the numpy array (shape: [num_stocks, num_days])
np.save(f"{TARGET_DIR}/ror.npy", ror_df.to_numpy())


(49, 2673)


## relation_file
| File_name | Shape | Description |
| ---- | ---- | ---- |
| relation_file (e.g. industry_classification.npy) | [num_stocks, num_stocks] | the relation matrix used in GCN layer|

### DT-industry

In [9]:
import yfinance as yf
import numpy as np
from pprint import pprint
from collections import defaultdict

# Create a Tickers object for multiple stocks in TWII_STOCKS
twii_tickers = yf.Tickers(" ".join(TWII_STOCKS))

# Build a dictionary to map each sector to a set of stocks in that sector
industry = defaultdict(set)
for stock in twii_tickers.symbols:
    ticker = twii_tickers.tickers[stock]
    # Extract sector information from ticker.info; assume it exists
    sector = ticker.info.get('sector')
    if sector is not None:
        industry[sector].add(stock)

# Print the industry classification dictionary for reference
pprint(dict(industry))

# Build the relation matrix based on industry classification
industry_classification = []
for stock in twii_tickers.symbols:
    ticker = twii_tickers.tickers[stock]
    # Get the sector of the current stock
    sector = ticker.info.get('sector')
    # Determine the number of stocks in this sector; if no sector info, default to 1
    sector_count = len(industry[sector]) if sector is not None else 1
    # For each stock in twii_tickers.symbols, assign weight = 1/sector_count if it belongs to the same sector, else 0.0
    relation = [1.0 / sector_count if other_stock in industry[sector] else 0.0 for other_stock in twii_tickers.symbols]
    industry_classification.append(relation)

# Save the relation matrix as a text file for inspection and as a numpy file for use in GCN
with open(f"{TARGET_DIR}/industry_classification.txt", "w") as f:
    for row in industry_classification:
        # Each value is formatted to two decimal places
        f.write(' '.join([f'{num:.2f}' for num in row]) + '\n')

# Convert the relation matrix to a numpy array with shape [num_stocks, num_stocks]
industry_classification = np.array(industry_classification)
print(industry_classification.shape)
np.save(f"{TARGET_DIR}/industry_classification.npy", industry_classification)


{'Basic Materials': {'2002.TW', '1326.TW', '1101.TW', '1301.TW', '1303.TW'},
 'Consumer Cyclical': {'2207.TW'},
 'Consumer Defensive': {'1216.TW', '2912.TW'},
 'Financial Services': {'2801.TW',
                        '2882.TW',
                        '2885.TW',
                        '2886.TW',
                        '2890.TW',
                        '5880.TW'},
 'Industrials': {'2603.TW'},
 'Technology': {'2301.TW',
                '2303.TW',
                '2308.TW',
                '2317.TW',
                '2327.TW',
                '2330.TW',
                '2345.TW',
                '2357.TW',
                '2379.TW',
                '2382.TW',
                '2395.TW',
                '3037.TW',
                '3711.TW'}}
(28, 28)


### DT-correlation

In [37]:
correlation_matrix = np.corrcoef(ror_df)
print(correlation_matrix.shape)
np.save(f"{TARGET_DIR}/industry_classification.npy", correlation_matrix)

(49, 49)
